<a href="https://colab.research.google.com/github/shv-om/Tracing-IP-address-and-details-of-unidentified-participants-while-academic-sessions/blob/main/CV_InnoFest_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install face_recognition

In [ ]:
import face_recognition
import cv2
import numpy as np
import os
import glob

In [ ]:
path_ = '/content/drive/MyDrive/CV_Hackathon/train_folder/'
# path = os.path.join(cur_direc, 'data/faces/')

list_of_files = [f for f in glob.glob(path_+'*.jpg')]

number_files = len(list_of_files)
names = list_of_files.copy()

In [ ]:
faces_encodings = []
faces_names = []

for i in range(number_files):
  globals()['image_{}'.format(i)] = face_recognition.load_image_file(list_of_files[i])
  globals()['image_encoding_{}'.format(i)] = face_recognition.face_encodings(globals()['image_{}'.format(i)])[0]
  faces_encodings.append(globals()['image_encoding_{}'.format(i)])
  
  # Create array of known names
  names[i] = names[i].replace(path_, "")  
  faces_names.append(names[i])

faces_names = list(set(faces_names))

In [ ]:
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

In [ ]:
test_list = [nf for nf in glob.glob('/content/drive/MyDrive/CV_Hackathon/test_folder/'+'*.jpg')]

for test_path in test_list:
  
  newname = os.path.basename(test_path)

  img = cv2.imread(test_path)

  small_frame = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
  rgb_small_frame = small_frame[:, :, ::-1]

  if process_this_frame:
    face_locations = face_recognition.face_locations( rgb_small_frame)
    face_encodings = face_recognition.face_encodings( rgb_small_frame, face_locations)
    face_names = []
    
    for face_encoding in face_encodings:
      matches = face_recognition.compare_faces(faces_encodings, face_encoding)
      name = "Unknown"
      face_distances = face_recognition.face_distance(faces_encodings, face_encoding)
      best_match_index = np.argmin(face_distances)
      
      if matches[best_match_index]:
        name = os.path.basename(faces_names[best_match_index]).split('.')[0]
        face_names.append(name)

  process_this_frame = not process_this_frame
  # Display the results
  for (top, right, bottom, left), name in zip(face_locations, face_names):
    top *= 2
    right *= 4
    bottom *= 5
    left *= 4
    
    # Draw a rectangle around the face
    cv2.rectangle(img, (left, top), (right, bottom), (40, 167, 69), 2)
    # Input text label with a name below the face
    cv2.rectangle(img, (left, bottom - 35), (right, bottom), (40, 167, 69), cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(img, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    # Display the resulting image

    newpath = '/content/drive/MyDrive/CV_Hackathon/test_folder/output/' + os.path.basename(newname)
    cv2.imwrite(newpath, img)